<a href="https://colab.research.google.com/github/dhwan-dj/S4pjt_NLP_koBERT/blob/main/kobert_sentiment_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 개요 : KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만드는 것인데, 공포, 놀람, 분노, 슬픔, 중립, 행복, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. 예를 들어 "앗 깜작이야!" 라는 문장을 입력하면 '놀람'이라는 클래스로 예측을 하도록 학습시키는 것
- 코드는 KoBERT 깃허브에 있는 네이버 영화평 이중분류 예시 코드를 바탕으로 작성
- 참고 블로그 (https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab)

# 1. Colab 환경설정

- 라이브러리, 모듈 설치, koBERT 모델 불러오기
- 예시 코드의 Dataset = '한국어 감정 정보가 포함된 단발성 대화'
- 본 코드의 Dataset : 감성대화말뭉치(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86)

<세팅 및 파라미터>  

    Python >= 3.6
    PyTorch >= 1.70
    Transformers = 3.0.2
    Colab
    batch size = 64
    epochs = 10

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689018 sha256=f1ca987926b3ee9574d2b7ec3812df0f61b5e48f0fadbfe0110f89c02cb3fa11
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=4143f462e4918c0b7bc0fb9d37379d5048f02f99eb14ce69b89a155020652b09
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [3]:
# 깃허브에서 KoBERT 파일들 다운로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-x8yds4si
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-x8yds4si
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf 의 kobert_tokenizer 폴더를 다운받는 코드.
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [5]:
import torch             # pytorch framework
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
#kobert
from kobert.utils import get_tokenizer
# https://github.com/SKTBrain/KoBERT/tree/master/kobert/utils 의 utils.py 파일에서 get_tokenizer 메서드를 불러오는 코드
from kobert.pytorch_kobert import get_pytorch_kobert_model
# https://github.com/SKTBrain/KoBERT/tree/master/kobert 의 pytorch_kobert.py 파일에서 get_pytorch_kobert_model 메서드를 불러오는 코드

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


- BERT는 이미 누군가가 학습해둔 모델을 사용한다(pre-trained model)는 것을 뜻한다. 따라서 사용하는 model과 tokenizer는 항상 mapping 관계여야 한다. 예를 들어서 U 팀이 개발한 BERT를 사용하는데, V팀이 개발한 BERT의 tokenizer를 사용하면 model은 텍스트를 이해할 수 없다. U팀의 BERT의 토크나이저는 '우리'라는 단어를 23번으로 int encoding하는 반면에, V라는 BERT의 tokenizer는 '우리'라는 단어를 103번으로 int encoding해 단어와 mapping 되는 정보 자체가 달라지기 때문이다. 
(https://hoit1302.tistory.com/159)

# 2. 데이터셋 로드, 전처리

In [9]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
train = pd.read_excel('/content/drive/MyDrive/data/sentiment_train.xlsx')
val = pd.read_excel('/content/drive/MyDrive/data/sentiment_val.xlsx')

In [11]:
df = pd.concat([train,val])

In [12]:
len(df) # 문장 = 58,271개

58271

In [13]:
df.sample(5)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
30183,30184,청소년,여성,가족관계,해당없음,기쁨,신이 난,오늘 아빠랑 여행가.,아버지와 여행을 계획하시는군요. 더 자세히 말씀해 주시겠어요?,바다로 가! 너무 좋아.,아버지와 함께 바다에 가신다니 정말 즐거울 것 같아요.,NaN,NaN
35587,35588,청소년,여성,학업 및 진로,해당없음,당황,고립된,나는 아직도 모르겠는데 학교 친구들은 벌써 미래희망이 있더라. 나 혼자 고립된 느낌이야.,친구분들과 사정이 달라서 혼자 소외된 느낌을 받으셨군요. 좋아하는 일은 없으신가요?,동물 키우는걸 좋아하지만 진로는 아니야. 시험공부 하기도 힘겨워서 생각 안 해봤어.,시험공부 하느라 진로에 대해 생각할 여유가 없으시군요.,NaN,NaN
1794,1795,청소년,남성,학교폭력/따돌림,해당없음,불안,두려운,나 너무 힘들어서 학교 가기 싫어. 친구들 때문에 무서워.,친구들이 어떨 때 가장 무섭나요?,빨리 빵 안 사 온다고 때릴 때 가장 무서워. 난 정말 빨리 간 건데.,빵 사 오라고 괴롭힐 때마다 정말 괴로우실 것 같아요.,그래도 용기 내서 학교는 가야겠지.,힘들어도 학교에 가겠다는 의지가 있으시군요.
231,232,청년,남성,"연애,결혼,출산",해당없음,분노,방어적인,아직 확신이 들지도 않고 결혼이 망설여져.,결혼을 망설이고 계시는군요. 왜 그렇게 생각을 하시는지 좀 더 자세히 얘기해 주실 ...,결혼 자금으로 돈을 충분히 모으지도 못했고 또 결혼 생활이 행복할지 잘 모르겠어.,결혼 후 생활이 행복할지 아직 확신이 없으시군요.,NaN,NaN
42060,42061,청년,여성,대인관계,해당없음,분노,방어적인,왜 난 하는 일마다 다 망치는 걸까? 난 정말 쓸모없는 인간인 것 같아.,쓸모가 없는 것 같아서 스스로 크게 실망하셨군요.,남들은 알아서 잘사는 것 같은데 나만 아무것도 못 하는 쓰레기 같아.,그동안 이런 마음이 들 때는 무엇을 하셨나요?,언니에게 하소연하면서 울었어. 언니는 내 말을 잘 들어줘.,언니와 대화하면서 속상한 마음이 사라졌으면 좋겠어요.


- 6번째 칼럼 '감정_대분류'를 라벨 삼아 입력문장의 감정을 예측하기로 한다.

In [14]:
df.감정_대분류.value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정_대분류, dtype: int64

In [15]:
# 주관적으로 긍정에서 부정으로 숫자 라벨을 매겨보았다.
df.loc[(df['감정_대분류'] == "기쁨"), 'sentiment'] = '0'    #기쁨 => 0
df.loc[(df['감정_대분류'] == "당황"), 'sentiment'] = '1'    #당황 => 1
df.loc[(df['감정_대분류'] == "불안"), 'sentiment'] = '2'    #불안 => 2
df.loc[(df['감정_대분류'] == "분노"), 'sentiment'] = '3'    #분노 => 3
df.loc[(df['감정_대분류'] == "슬픔"), 'sentiment'] = '4'    #슬픔 => 4
df.loc[(df['감정_대분류'] == "상처"), 'sentiment'] = '5'    #상처 => 5

In [16]:
# 문장과 문장의 감정을 리스트로 저장
data_list = []
for q, label in zip(df['사람문장1'], df['sentiment']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [17]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '3']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.', '3']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.', '2']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.', '5']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.', '5']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.', '4']
['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.', '1']


In [38]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 41.3 MB/s eta 0:00:00


In [70]:
from konlpy.tag import Okt
from collections import Counter
import nltk, re

nltk.download('stopwords')
stopwords = ['같아', '스러워','하고','만','해서','도','에게','안','못','너무','내','나','에서','해','로','한테','했는데',
             '을', '에', '를', '은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한',
             '했어', '하는', '게', '들어', '말', '으로','이야', '오늘','요즘','이번','할','봐','돼', '때문', '잘','정말'
             ]
okt = Okt()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
def preprocess_sentence(sentence):
    # 구두점 제거
    sentence = re.sub(r'[^\w]', ' ', sentence)

    # 소문자로 변환
    sentence = sentence.lower()

    # 토큰으로 분리
    tokens = okt.morphs(sentence)

    # stopwords 제거
    tokens = [token for token in tokens if not token in stopwords]

    return tokens

def count_tokens(dataset, label):
    # 데이터셋에서 label이 label인 문장만 추출
    sentences = [dataset[i][0] for i in range(len(dataset)) if dataset[i][1] == str(label)]
    
    # 불용어 제거하고 토큰으로 분리
    tokens = [preprocess_sentence(sentence) for sentence in sentences]
    
    # combine all the tokens into a single list
    all_tokens = [token for sublist in tokens for token in sublist]
    
    # 토큰들의 빈도수 측정
    counter = Counter(all_tokens)
    
    return counter

# label 0의 top 10 tokens (기쁨 => 0, 당황 => 1, 불안 => 2, 분노 => 3, 슬픔 => 4, 상처 => 5)
counter = count_tokens(data_list, 0)
top_10 = counter.most_common(10)
print("기쁨 top_10 : ", top_10)

기쁨 top_10 :  [('친구', 885), ('기뻐', 708), ('정말', 628), ('잘', 614), ('좋아', 548), ('우리', 497), ('회사', 428), ('다행', 409), ('기분', 405), ('사람', 370)]


In [68]:
# label 1의 top 10 tokens (기쁨 => 0, 당황 => 1, 불안 => 2, 분노 => 3, 슬픔 => 4, 상처 => 5)
counter = count_tokens(data_list, 1)
top_10 = counter.most_common(10)
print("당황 top_10 : ", top_10)

당황 top_10 :  [('친구', 1872), ('당황', 928), ('사람', 710), ('회사', 472), ('아내', 446), ('돈', 433), ('부모님', 432), ('일', 427), ('죄책감', 424), ('남편', 410)]


In [67]:
# label 1의 top 10 tokens (기쁨 => 0, 당황 => 1, 불안 => 2, 분노 => 3, 슬픔 => 4, 상처 => 5)
counter = count_tokens(data_list, 2)
top_10 = counter.most_common(10)
print("불안 top_10 : ", top_10)

불안 top_10 :  [('친구', 1316), ('걱정', 1040), ('불안해', 883), ('사람', 520), ('회사', 514), ('일', 512), ('해야', 496), ('돈', 482), ('스트레스', 463), ('하는데', 422)]


In [71]:
counter = count_tokens(data_list, 3)
top_10 = counter.most_common(10)
print("분노 top_10 : ", top_10)

분노 top_10 :  [('친구', 1524), ('화가', 1282), ('짜증', 627), ('남편', 614), ('사람', 598), ('돈', 520), ('아내', 517), ('일', 514), ('자꾸', 504), ('고', 465)]


In [72]:
counter = count_tokens(data_list, 4)
top_10 = counter.most_common(10)
print("슬픔 top_10 : ", top_10)

슬픔 top_10 :  [('슬퍼', 1406), ('친구', 1405), ('생각', 524), ('사람', 497), ('일', 486), ('회사', 474), ('돈', 467), ('아내', 464), ('부모님', 458), ('눈물', 446)]


In [ ]:
counter = count_tokens(data_list, 5)
top_10 = counter.most_common(10)
print("상처 top_10 : ", top_10)

# 3. Train과 Test Set 준비

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.15, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

# 4. koBERT의 입력 데이터 형태로 만들기
- 모델의 입력으로 활용하기 위해 토큰화, 라벨과 문장의 인덱스 인코딩, 패딩 등이 필요하다.
- 예시 코드의 입력데이터 형태로 바꿔주는 클래스 활용

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
      transform = nlp.data.BERTSentenceTransform(
          bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)   # BERTSentenceTransform 으로 토큰화, 패딩
      
      self.sentences = [transform([i[sent_idx]]) for i in dataset]
      #self.labels = [np.int32(i[label_idx]) for i in dataset]
      self.labels = [str(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
      return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
      return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERTTokenizer와 위의 Class로 Tokenize, Padding
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

- 토큰화와 패딩이 잘 이루어졌을까?
- 데이터는 3개의 array인데, 첫번째는 패딩된 시퀀스, 두번째는 길이와 타입, 세번째는 어텐션 마스크 시퀀스라고 한다.
- BERT에 데이터가 입력되었을 때 어텐션 함수가 적용되어 연산이 된다. 이때 1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터가 있어야 하는데 그게 바로 어텐션 마스크 시퀀스 라고 한다. (아직 이해x)

In [ ]:
data_train[0]

In [ ]:
# torch 형식의 dataset을 만들어준다.
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# 5. koBERT 학습모델 만들기
- num_classes = 6 (6개의 감정 클래스)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. BERT model 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

- Epochs 5 학습은 약 47분이 걸렸고, train dataset에 대해서는 0.747, test dataset에 대해서는 0.594의 정확도를 기록했다.

- 이거 좀 낮은데;;

- 참고블로그에서는 정확도가 높았는데, 이에 대한 설명 :
train dataset에 대해서는 0.979, test dataset에 대해서는 0.918의 정확도를 기록
이렇게 높은 정확도를 기록하는 이유는 바로 데이터셋에 있다. 진행하고 있는 프로젝트에서 해당 프로젝트의 성격에 맞게 3만 7천 여개 정도의 데이터를 직접 재분류하고 있는데, 사람이 생각하기에는 분명히 같은 내용으로 인지되는 문장이지만 사소한 단어를 한 두개 없애서, 어떨 때는 중요한 단어를 제거해서, 짧게 잘라서, 길게 늘여뜨려서, 문장 부호를 다르게, 감탄사를 추가해서 등등 중복인 듯 중복 아닌 데이터로 학습시켰기 때문에 이렇게 높은 정확도가 나온 것이라 판단된다.
(https://hoit1302.tistory.com/159)

In [ ]:
from typing import Tuple
import torch

class F1Score:
    """
    Class for f1 calculation in Pytorch.
    """
    def __init__(self, average: str = 'weighted'):
        """
        Init.

        Args:
            average: averaging method
        """
        self.average = average
        if average not in [None, 'micro', 'macro', 'weighted']:
            raise ValueError('Wrong value of average parameter')

    @staticmethod
    def calc_f1_micro(predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 micro.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """
        true_positive = torch.eq(labels, predictions).sum().float()
        f1_score = torch.div(true_positive, len(labels))
        return f1_score

    @staticmethod
    def calc_f1_count_for_label(predictions: torch.Tensor,
                                labels: torch.Tensor, label_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Calculate f1 and true count for the label

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels
            label_id: id of current label

        Returns:
            f1 score and true count for label
        """
        # label count
        true_count = torch.eq(labels, label_id).sum()

        # true positives: labels equal to prediction and to label_id
        true_positive = torch.logical_and(torch.eq(labels, predictions),
                                          torch.eq(labels, label_id)).sum().float()
        # precision for label
        precision = torch.div(true_positive, torch.eq(predictions, label_id).sum().float())
        # replace nan values with 0
        precision = torch.where(torch.isnan(precision),
                                torch.zeros_like(precision).type_as(true_positive),
                                precision)

        # recall for label
        recall = torch.div(true_positive, true_count)
        # f1
        f1 = 2 * precision * recall / (precision + recall)
        # replace nan values with 0
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1).type_as(true_positive), f1)
        return f1, true_count

    def __call__(self, predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 score based on averaging method defined in init.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """

        # simpler calculation for micro
        if self.average == 'micro':
            return self.calc_f1_micro(predictions, labels)

        f1_score = 0
        for label_id in range(1, len(labels.unique()) + 1):
            f1, true_count = self.calc_f1_count_for_label(predictions, labels, label_id)

            if self.average == 'weighted':
                f1_score += f1 * true_count
            elif self.average == 'macro':
                f1_score += f1

        if self.average == 'weighted':
            f1_score = torch.div(f1_score, len(labels))
        elif self.average == 'macro':
            f1_score = torch.div(f1_score, len(labels.unique()))

        return f1_score

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
errors = 0
for _ in range(10):
    labels = torch.randint(1, 10, (4096, 100)).flatten()
    predictions = torch.randint(1, 10, (4096, 100)).flatten()
    labels1 = labels.numpy()
    predictions1 = predictions.numpy()

    for av in ['micro', 'macro', 'weighted']:
        f1_metric = F1Score(av)
        my_pred = f1_metric(predictions, labels)
        
        f1_pred = f1_score(labels1, predictions1, average=av)
        
        if not np.isclose(my_pred.item(), f1_pred.item()):
            print('!' * 50)
            print(f1_pred, my_pred, av)
            errors += 1

if errors == 0:
    print('No errors!')

# 7. 새로운 문장 테스트
- 새로운 문장을 koBERT의 입력 형식으로 바꿔주는 predict 함수를 정의한다.

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

# **Result**
- 학습데이터 정확도
- 테스트데이터 정확도
- F1, Precision, Recall 점수를 계산
- 새로운 문장에 대한 정확도

- 한계점

- 개선점

- 기대효과